In [3]:
import numpy as np
import pandas as pd

In [5]:
import sys
#reload(sys)
# sys.setdefaultencoding('big5')

In [52]:
# create simulated data

# number of row or column
n_row = 10
n_col = 100

data_ = np.abs(np.random.rand(n_row, n_col)) * 100
data_ = data_.astype(np.int)

data = pd.DataFrame(
    data_,
    columns=['age_' + str(i) for i in range(n_col)],
    index=['region_' + str(i) for i in range(n_row)]
)

data

,age_0,age_1,age_2,age_3,age_4,age_5,age_6,age_7,age_8,age_9,...,age_90,age_91,age_92,age_93,age_94,age_95,age_96,age_97,age_98,age_99
region_0,41,79,45,4,4,40,52,49,23,96,...,78,2,57,28,45,8,68,54,23,95
region_1,91,21,23,83,24,66,10,30,94,19,...,42,64,71,20,91,94,22,59,70,50
region_2,67,2,34,74,6,49,66,69,36,75,...,76,35,53,84,67,39,96,83,73,56
region_3,19,74,21,32,94,1,5,41,6,47,...,97,85,38,13,77,85,0,49,7,86
region_4,4,25,87,99,1,81,88,88,87,68,...,29,86,31,65,4,2,70,81,53,0
region_5,22,63,83,24,85,71,60,87,79,97,...,92,29,43,8,33,7,90,65,86,30
region_6,74,52,77,99,38,77,12,22,49,82,...,71,51,6,84,90,3,95,99,14,18
region_7,61,54,92,68,59,60,41,66,82,67,...,40,71,17,33,98,33,3,75,75,79
region_8,50,74,33,42,10,40,69,33,54,0,...,4,76,85,96,17,45,0,28,81,72
region_9,23,52,23,66,16,59,12,13,12,95,...,72,7,22,74,71,28,99,58,69,88


### preprocess

In [70]:
# transfer chr into int
def parse_age(name):
    if np.iterable(name) and not isinstance(name, str):
        return [parse_age(i) for i in name]
    
    if not isinstance(name, str):
        raise ValueError
        
    return int(name.split('_')[1])

In [86]:
# row names
index_name = data.index

# column names
column_name = data.columns
ages = parse_age(column_name)

# elements in `data`
values = data.values

In [88]:
# the info in the above block

print len(index_name), len(column_name)
print values.shape

10 100
(10L, 100L)


### limiting data range

In [118]:
def parse_location(array, target):
    """
    Assumption
    ----------
    `array` is an unique and sorted array
    
    Return
    ------
    index: list or a value
        * the index in array where array[index] >= target
    """
    if not np.all(np.sort(array) == array) or \
       not len(np.unique(array) == len(array)):
        raise ValueError('please check the detail in docstring for ensuring input value')
        
    r = np.searchsorted(array, target)
    r[r == len(array)] = -1
    return r

In [128]:
check_ages = [0, 15, 65, 100, 200, 300, 400]

print 'result of parse_location', parse_location(ages, check_ages)

datas = []

for start_age, end_age in zip(check_ages[:-1], check_ages[1:]):
    value = values[:, start_age:end_age]
    print value.shape
    datas.append(value)

result of parse_location [ 0 15 65 -1 -1 -1 -1]
(10L, 15L)
(10L, 50L)
(10L, 35L)
(10L, 0L)
(10L, 0L)
(10L, 0L)


In [130]:
check_ages = [15, 65]

I = parse_location(ages, check_ages)

datas = np.split(values, I, axis=-1)
for i in datas:
    print i.shape

(10L, 15L)
(10L, 50L)
(10L, 35L)


### main work

In [134]:
# peek of datas
for i in datas:
    print i.shape

(10L, 15L)
(10L, 50L)
(10L, 35L)


#### method 1

In [143]:
r = []
for i in datas:
    v = np.sum(i, axis=-1)
    print v.shape
    print v[:, None].shape
    
    r.append(v[:, None])

(10L,)
(10L, 1L)
(10L,)
(10L, 1L)
(10L,)
(10L, 1L)


In [144]:
r = np.hstack(r)
print r.shape

pd.DataFrame(
    r,
    index=index_name
)

(10L, 3L)


,0,1,2
region_0,614,1968,1669
region_1,824,2853,1923
region_2,765,2523,1870
region_3,536,2907,1511
region_4,811,2521,1750
region_5,857,2453,1442
region_6,741,2313,1560
region_7,821,2505,1954
region_8,713,2919,1888
region_9,566,2468,1936


#### method 2

In [149]:
r = np.hstack([np.sum(i, axis=-1)[:, None] for i in datas])
pd.DataFrame(
    r,
    index=index_name
)

,0,1,2
region_0,614,1968,1669
region_1,824,2853,1923
region_2,765,2523,1870
region_3,536,2907,1511
region_4,811,2521,1750
region_5,857,2453,1442
region_6,741,2313,1560
region_7,821,2505,1954
region_8,713,2919,1888
region_9,566,2468,1936


### xlsx process 

In [6]:
import re
import os
import csv
import xlrd
import openpyxl

In [80]:
def is_age(x):
    if x is None:
        return False
    r = max(len(re.findall(r'\u6b72', x)), len(re.findall(r'歲', x)))
    return r > 0

def is_age_row(r):
    return np.any([is_age(i) for i in r])

def parse_age(x):
#     print (x)
#     print (x.encode('ascii'))
    x = x.replace(r'\u6b72', "")
    x = x.replace(r'歲', "")
    x = x.replace(r'\uff5e', "")
    x = x.replace(r'～', "")
    x = x.split(' ')
    x = [i for i in x if len(i) > 0]
    x = [int(i) for i in x]
    return x

def is_number_string(x):
    try:
        parse_number_string(x)
        return True
    except:
        return False
    
def is_number_row(r):
    return np.any([is_number_string(i) for i in r])

def parse_number_string(number):
    x = number.replace(r'\u2014', '0\n')
    x = number.replace('—', '0\n')
    x = x.split('\n')
    x = [i for i in x if len(i) > 0]
    
    assert(len(x) == 3)
    
    def f(x):
        r = ''
        for i in x.split(','):
            r = r + i
        return int(r)

    return [f(i) for i in x]

In [40]:
# os.listdir(file_folder)

In [73]:
file_folder = r'/Users/nadia/Desktop'
file_name = '10810.xlsx'

In [74]:
workbook = openpyxl.load_workbook(os.path.join(file_folder, file_name))
worksheet = workbook.worksheets[0]
workbook.close()

In [75]:
values = []

for i in worksheet.values:
    values_ = []
    for j in i:
        values_.append(j)
    values.append(values_)

In [83]:
values[7]

[10018000,
 None,
 '總    計',
 None,
 '計',
 448472,
 None,
 23923,
 None,
 3793,
 None,
 4328,
 None,
 4769,
 None,
 5268,
 None,
 5765,
 None,
 27504,
 None,
 5837,
 None,
 5559,
 None,
 6236,
 None,
 None]

In [79]:
# at the end of this block, A[0] should be digit_row, rest of rows should be number_row

A = [i for i in values if (is_age_row(i) or is_number_row(i))]
splits = [i for i, j in enumerate(A) if is_age_row(j)][::2]
splits = [i for i in splits if i != 0]
A = np.hstack(np.split(A, splits))
A = A[1:]

# handle age row
age_row = [parse_age(i)[0] if is_age(i) else None for i in A[0]]
age_ind = [i for i, j in enumerate(age_row) if j is not None]
num_ind = [i for i, j in enumerate(A[1]) if is_number_string(j)]

names = A[1:, 0]
names = np.repeat(names, 3).tolist()

sex_names = [u'\u8a08\n\u7537\n\u5973'] * 4

r = []
for i in sex_names:
    r.extend(i.split('\n'))
sex_names = r

公開類
次月5日前編送
月    報
月統計報表之3
新竹市各區    現住人口數按性別及年齡分
中華民國108年11月底
RRRP03360
區域代碼
區域別
性別
總計
0～4歲
5～9歲
合計
0歲
1歲
2歲
3歲
4歲
合計
5歲
6歲
7歲
10018000


TypeError: expected string or bytes-like object

In [70]:
filter_A = A[1:, num_ind]

data = []
for i in filter_A.tolist():
    data_ = []
    for j in i:
        data_.append(parse_number_string(j))      
    r_ = np.vstack(data_).T
    data.append(r_)
data = np.vstack(data)

In [71]:
# prepare labels
mask = np.r_[[is_age(i) for i in A[0, num_ind]]]

labels = [''] * len(mask)

I = np.where(mask)[0]
J = ['age_' + str(parse_age(i)[0]) for i in A[0, num_ind][mask]]

for i, j in zip(I, J):
    labels[i] = j

I = np.where(~mask)[0]
J = ['total_age' + str(i) for i in range(len(I))]

for i, j in zip(I, J):
    labels[i] = j
    
labels[0] = 'total'
labels = ['area', 'gender'] + labels

In [72]:
output_name = file_name.replace('.xlsx', '.csv')

with open(os.path.join(file_folder, output_name), 'w', encoding='Big5') as fd:
    spamwriter = csv.writer(fd, delimiter=',')
    spamwriter.writerow(labels)
    
    for i, j, k in zip(names, sex_names, data):
        spamwriter.writerow([i] + [j] + k.tolist())
    

In [408]:
# names = A[2:, 0]    # skip total
# D = A[2:, age_ind]  # skip total

# D_data = {}

# for n, i in enumerate(D):
#     arr = np.array([parse_number_string(j)[1:] for j in i]).T
#     assert(arr.shape[0] == 2) 
#     D_data_ = {
#         'male': arr[0],
#         'female': arr[1],
#         'ages': np.r_[age_row][age_ind]
#     }
    
#     D_data[names[n]] = D_data_

In [409]:
# # show array
# for d in D_data.values():
#     r = pd.DataFrame(
#         np.vstack([d['male'], d['female']]),
#         columns=['age_' + str(i) for i in d['ages']],
#         index=['male', 'female']
#     )
#     print r

        age_0  age_1  age_2  age_3  age_4  age_5  age_6  age_7  age_8  age_9  \
male      607    775    746    836    912    844    969    947    729    730   
female    563    685    716    816    896    806    954    839    677    691   

        ...  age_90  age_91  age_92  age_93  age_94  age_95  age_96  age_97  \
male    ...      91      82      82      67      48      42      19      10   
female  ...     113      91      63      53      43      34      28      12   

        age_98  age_99  
male        14       7  
female       7       7  

[2 rows x 100 columns]
        age_0  age_1  age_2  age_3  age_4  age_5  age_6  age_7  age_8  age_9  \
male      960   1164   1289   1518   1679   1613   1798   1778   1480   1561   
female    973   1119   1261   1405   1538   1484   1707   1564   1383   1457   

        ...  age_90  age_91  age_92  age_93  age_94  age_95  age_96  age_97  \
male    ...     113     111      86      81      58      39      32      19   
female  ...     149    

In [ ]:
# def export_results(data, region=None, sex=None, age_range=None):
#     if region is not None and region not in data.keys():
#         raise ValueError
        
#     if region is not None:
#         return export_results(
#             {region: data[region]},
#             sex=sex,
#             age_range=age_range
#         )
    
#     if sex is not None and sex not in ['female', 'male']:
#         raise ValueError
        
#     if age_range is not None:
#         assert(isinstance(age_range, list))
#         assert(np.all([len(i) == 2 for i in age_range]))
    
#     def f(x, s_age, e_age):
#         n = ['male', 'female']
#         if sex is not None:
#             n = [sex]
#         X = np.vstack([n[i] for i in n])
        
#         s_ = np.searchsorted(x['ages'], s_age)
#         e_ = np.searchsorted(x['ages'], e_age)
#         pd
#         return X[:, s_:e_]
    
#     if age_range is not None:
        
        
        